In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import spacy

# tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
# model_nlp = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

# ner = pipeline("ner", model=model_nlp, tokenizer=tokenizer, grouped_entities=True)

In [6]:
! python -m spacy download ru_core_news_lg

  Using cached https://github.com/explosion/spacy-models/releases/download/ru_core_news_lg-3.7.0/ru_core_news_lg-3.7.0-py3-none-any.whl (513.4 MB)
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [7]:
ner = spacy.load('ru_core_news_lg')

In [8]:
sentiment = pipeline(model="MonoHime/rubert-base-cased-sentiment-new")

config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
text = 'Дэниэл Томпсон поступил в НИТУ МИСИС в Москве и купил Apple iPhone 15'

In [10]:
%%time
result = dict()

for ent in ner(text).ents:
    result[ent.text] = ent.label_

result

CPU times: user 23.7 ms, sys: 1.42 ms, total: 25.1 ms
Wall time: 36.9 ms


{'Дэниэл Томпсон': 'PER', 'НИТУ МИСИС': 'ORG', 'Москве': 'LOC', 'Apple': 'ORG'}

In [11]:
%%time
print(sentiment(text)[0]['label'])

NEUTRAL
CPU times: user 223 ms, sys: 8.91 ms, total: 231 ms
Wall time: 186 ms


In [2]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

MODEL_NAME = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs):

    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

text = """Высота башни составляет 324 метра (1063 фута), примерно такая же высота, как у 81-этажного здания, и самое высокое сооружение в Париже. Его основание квадратно, размером 125 метров (410 футов) с любой стороны. Во время строительства Эйфелева башня превзошла монумент Вашингтона, став самым высоким искусственным сооружением в мире, и этот титул она удерживала в течение 41 года до завершения строительство здания Крайслер в Нью-Йорке в 1930 году. Это первое сооружение которое достигло высоты 300 метров. Из-за добавления вещательной антенны на вершине башни в 1957 году она сейчас выше здания Крайслер на 5,2 метра (17 футов). За исключением передатчиков, Эйфелева башня является второй самой высокой отдельно стоящей структурой во Франции после виадука Мийо."""

print(summarize(text, n_words=10))

Французская Эйфелева башня достигла высоты 300 метров.
